In [23]:
# Problem 1: Geocode shopping centers
# Import the geocoding tool
import pandas as pd


# file path
fp = "/Users/luyc/Desktop/Automated GIS python/exercise 3/shopping_center.txt"

# read data
data = pd.read_csv(fp,sep=';')
print(len(data))
print(data.head())


7
     id       name                                            addr
0  1000       Itis            Itäkatu 1-7, 00930 Helsinki, Finland
1  1001      Forum  Mannerheimintie 14-20, 00100 Helsinki, Finland
2  1002  Iso-omena           Piispansilta 11, 02230 Espoo, Finland
3  1003      Sello       Leppävaarankatu 3-9, 02600 Espoo, Finland
4  1004      Jumbo      Vantaanportinkatu 3, 01510 Vantaa, Finland


In [25]:
from geopandas.tools import geocode

# Geocode addresses using Nominatim.
geo = geocode(data['addr'], provider='nominatim', user_agent='luyc', timeout=4)

# Change projection system
geo = geo.to_crs("EPSG:3879")
print(geo.head())

# table join
join = geo.join(data)

# output path
outfp = "/Users/luyc/Desktop/Automated GIS python/exercise 3/shopping_center.shp"
join.to_file(outfp)

                           geometry  \
0  POINT (25504598.602 6677662.109)   
1  POINT (25496573.542 6672878.360)   
2  POINT (25485510.634 6672029.255)   
3  POINT (25489571.409 6678401.654)   
4  POINT (25497943.932 6686656.982)   

                                             address  
0  Kauppakeskus Itis, Itäkatu, Itäkeskus, Vartiok...  
1  Salaattiasema, Mannerheimintie, Keskusta, Kluu...  
2  11, Piispansilta, Nuottaniemi, Piispansilta, E...  
3  3-9, Leppävaarankatu, Etelä-Leppävaara, Leppäv...  
4  Stockmann, Vantaanportinkatu, Pakkala, Aviapol...  


In [31]:
# Question 2: Create buffers around shopping centers
from shapely.geometry import Point,Polygon
import geopandas as gpd

# create buffer
join['buffer'] = join['geometry'].buffer(1500)
print(join.head())
print(type(join.at[0,'buffer']))

# Check the areas of your buffers in km^2
print(round(gpd.GeoSeries(join["buffer"]).area / 1000000))

# Replace the values in geometry column with the values of buffer column
join['geometry'] = join['buffer']

# drop out unnecessary columns from the geodataframe
buffer = join[['id','name','geometry']]
print(buffer.head())

                                            geometry  \
0  POLYGON ((25512091.374 6677956.515, 25512092.1...   
1  POLYGON ((25504066.315 6673172.766, 25504067.1...   
2  POLYGON ((25493003.406 6672323.661, 25493004.1...   
3  POLYGON ((25497064.182 6678696.060, 25497064.9...   
4  POLYGON ((25505436.705 6686951.388, 25505437.4...   

                                             address    id       name  \
0  Kauppakeskus Itis, Itäkatu, Itäkeskus, Vartiok...  1000       Itis   
1  Salaattiasema, Mannerheimintie, Keskusta, Kluu...  1001      Forum   
2  11, Piispansilta, Nuottaniemi, Piispansilta, E...  1002  Iso-omena   
3  3-9, Leppävaarankatu, Etelä-Leppävaara, Leppäv...  1003      Sello   
4  Stockmann, Vantaanportinkatu, Pakkala, Aviapol...  1004      Jumbo   

                                             addr  \
0            Itäkatu 1-7, 00930 Helsinki, Finland   
1  Mannerheimintie 14-20, 00100 Helsinki, Finland   
2           Piispansilta 11, 02230 Espoo, Finland   
3       Lepp

In [ ]:
# Problem 3: How many people live near shopping centers?
